In [41]:
"""
2. ConversationBufferWindowMemory
#@tag:notebookOutputLayout , use to format ouput
"""

'\n2. ConversationBufferWindowMemory\n'

In [42]:
# load openai key
from dotenv import load_dotenv,find_dotenv
_ = load_dotenv(find_dotenv())

In [50]:
# create chat model
from langchain.chat_models import ChatOpenAI
chat_model = ChatOpenAI(temperature=0.0,max_tokens=1000)

In [66]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain

window_memory = ConversationBufferWindowMemory(
    k=2
)  # remember recent 2 conversation pairs(input,output)

conversation = ConversationChain(memory=window_memory, llm=chat_model, verbose=True)


In [67]:
print(conversation.predict(input="how long will Sun live? Answer in 50 words or less"))
print(f"memory is : {window_memory.buffer}")
print(conversation.predict(input="How fast can a mass of 70Kgs travel realistically? Answer in 50 Words or less"))
print(f"memory is : {window_memory.buffer}")
print(conversation.predict(input="What are consequenses of humanity on earth? Answer in 50 words or less"))
print(f"memory is : {window_memory.buffer}")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: how long will Sun live? Answer in 50 words or less
AI:

> Finished chain.
The Sun is currently about 4.6 billion years old and is expected to live for another 5 billion years. However, as it ages, it will expand into a red giant and eventually shed its outer layers, becoming a white dwarf. So, in total, the Sun's lifespan is estimated to be around 10 billion years.
memory is : Human: how long will Sun live? Answer in 50 words or less
AI: The Sun is currently about 4.6 billion years old and is expected to live for another 5 billion years. However, as it ages, it will expand into a red giant and eventually shed its outer layers, becoming a white dwarf. So, in tot

 # Conclusion:
 
 ConversationBufferWithWindow maintains a window of most recent `k` conversations between human and AI.
 Each conversation is of pair(human,AI). In real applications `k` should be sufficiently large.

 Prevents memory to grow out of context.

# ConversationTokenBufferMemory

Used to save memory on tokens used , since LLMs use tokens as cost measure and context limit

In [69]:
from langchain.memory import ConversationTokenBufferMemory

chat_model = ChatOpenAI(temperature=0.0)

memory = ConversationTokenBufferMemory(llm=chat_model,max_token_limit=50)

memory.save_context({"input":"Are Platypus mammals?"},{"output":"They are freak of nature"})
memory.save_context({"input":"Is AI overtaking world?"},{"output":"Seems so."})
memory.save_context({"input":"Can capitalism be controlled?"},{"output":"not unless humans give up on greed"})


print(memory.buffer)

Human: Is AI overtaking world?
AI: Seems so.
Human: Can capitalism be controlled?
AI: not unless humans give up on greed


# Conclusion

Only last `max_token_limit` length of tokens are kept in memory (most recent) and given to LLM as context. 
Does not necessarylity keep (Human,AI) in pairs

# ConversationSummaryMemory

Use LLM to generate summary of conversation so far , and use it as memory



In [70]:
from langchain.memory import ConversationSummaryBufferMemory

# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."


memory = ConversationSummaryBufferMemory(llm=chat_model, max_token_limit=100) # if more tokens all text , if less than summarized text
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

print(memory.load_memory_variables({}))

{'history': 'System: The human and AI exchange greetings. The human asks about the schedule for the day. The AI provides a detailed schedule, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of bringing a laptop to showcase the latest LLM demo during the lunch meeting.'}


In [71]:
conversation = ConversationChain(
    llm=chat_model, 
    memory = memory,
    verbose=True
)

conversation.predict(input="How Can I convince the team about AI revolution?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI exchange greetings. The human asks about the schedule for the day. The AI provides a detailed schedule, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of bringing a laptop to showcase the latest LLM demo during the lunch meeting.
Human: How Can I convince the team about AI revolution?
AI:

> Finished chain.


"To convince the team about the AI revolution, it's important to present a compelling case that highlights the potential benefits and opportunities it can bring to the company. Here are a few strategies you can consider:\n\n1. Educate them about the current AI landscape: Start by providing an overview of the advancements in AI technology and how it is being successfully implemented in various industries. Share examples of companies that have embraced AI and the positive impact it has had on their operations, productivity, and profitability.\n\n2. Highlight the competitive advantage: Emphasize how adopting AI can give your company a competitive edge in the market. Discuss how AI can automate repetitive tasks, improve decision-making processes, and enhance customer experiences. Show them how AI can help the company stay ahead of the curve and outperform competitors.\n\n3. Address concerns and misconceptions: Many people have concerns about AI, such as job displacement or ethical implicat

In [72]:
memory.load_memory_variables({})

{'history': 'System: The human and AI exchange greetings and discuss the schedule for the day, which includes a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of bringing a laptop to showcase the latest LLM demo during the lunch meeting. The human then asks how to convince the team about the AI revolution. The AI provides strategies, including educating them about the current AI landscape, highlighting the competitive advantage, addressing concerns and misconceptions, showcasing successful AI projects, and offering training and support. The AI encourages tailoring the approach to the specific needs and concerns of the team.'}

## Conclusion

ConversationSummaryMemory stores most recent AI response with token limit , and all previous conversation detail goes into `summary` key of memory.

Example : 
- Two conversation took place and token limit is reached.
- next conversation , first memory is summarized and placed in system text.
- Then left over tokens are taken and used to answer chat request.

Two other types of useful memories to look into :

- Vector data memory
- Entity memory 

